BiLSTM序列标注---盲汉翻译

In [1]:
import torch
import torch.nn as nn
import numpy as np

In [2]:
word_to_ix = np.load("word_to_ix.npy",allow_pickle=True)
tag_to_ix = np.load("tag_to_ix.npy",allow_pickle=True)

In [3]:
tag_to_ix

array({'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, ',': 5, '.': 6, '?': 7, '!': 8},
      dtype=object)

In [191]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix.item()[w.encode() ] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

f = open("./Testing data.txt", "r",encoding='UTF8') # r", encoding='UTF8')
#计数 数一共有多少行
lines = f.readlines()
testing_data = []
for index, line in enumerate(lines):
    line = lines[index].rstrip().split()
    # data = prepare_sequence(line,word_to_ix)
    # print(type(data))
    testing_data.append(line)
 

In [192]:
import torch.nn.functional as F
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # LSTM以word_embeddings作为输入, 输出维度为 hidden_dim 的隐藏状态值
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # 线性层将隐藏状态空间映射到标注空间
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # 一开始并没有隐藏状态所以我们要先初始化一个
        # 关于维度为什么这么设计请参考Pytoch相关文档
        # 各个维度的含义是 (num_layers*num_directions, batch_size, hidden_dim)
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores
    def predict(self,data,word_to_ix,tag_to_ix):
        # The input data is 2 dimensional list.
        self.zero_grad()
        result = []
        for i in range(len(data)):
            pre_data = data[i]
            pre_data = prepare_sequence(pre_data,word_to_ix)
            self.hidden = self.init_hidden()
            tag_scores = self(pre_data)
            tag_scores = torch.argmax(tag_scores, dim=-1)
            def decode(tag_scores,to_ix):
                to_ix = {v: k for k, v in to_ix.item().items()}
                return [to_ix[w.item()] for w in tag_scores]
            tag_scores = decode(tag_scores,tag_to_ix)
            result.append(tag_scores)
        return result
new_m = LSTMTagger(60,130,len(word_to_ix.item()),len(tag_to_ix.item()))
new_m.load_state_dict(torch.load('LSTMTagger[130,60].pt'))
new_m.predict(testing_data,word_to_ix,tag_to_ix)

[['3', '2', '3', '0', '4', '4', '0'], ['4', '3']]